# Importing Animation Samples and Preparing ZeroEGGS Subsamples

In [ ]:
# download zeroeggs zip files: https://github.com/ubisoft/ubisoft-laforge-ZeroEGGS.git
!wget https://github.com/ubisoft/ubisoft-laforge-ZeroEGGS/raw/main/data/Zeggs_data.zip
!wget https://github.com/ubisoft/ubisoft-laforge-ZeroEGGS/raw/main/data/Zeggs_data.z01
!wget https://github.com/ubisoft/ubisoft-laforge-ZeroEGGS/raw/main/data/Zeggs_data.z02

# extract split zip files
!7z x Zeggs_data.zip

# download subsample times: https://github.com/sinansonlu/animation-personality.git
!wget https://github.com/sinansonlu/animation-personality/raw/main/zeroeggs_subsample_times.zip

# extract subsample times zip file
!7z x zeroeggs_subsample_times.zip

In [ ]:
# install blender for cutting zeroeggs samples
!pip install bpy

In [ ]:
import bpy
import os
from glob import glob
from natsort import natsorted

# directory for zeroeggs bvh files
directory = 'clean/'

# text files for subsample times
txt_files = natsorted(glob(os.path.join('', '*.txt')))

# save directory for subsamples
savepath = 'zeroeggs_subsamples/'
!mkdir zeroeggs_subsamples

partcount = 0 # indicates the index of subsample
mainbvhindex = 1 # indicates the original sample where this subsample comes from

for txtfile in txt_files:
    # remove any object in scene
    bpy.ops.object.select_all(action='SELECT')
    bpy.ops.object.delete()

    # bvh file name ends with bvh instead of txt
    bvhfile = txtfile.replace('txt','bvh')

    # import bvh animation
    bpy.ops.import_anim.bvh(filepath=directory+bvhfile, filter_glob='*.bvh', target='ARMATURE', global_scale=1.0, frame_start=1, use_fps_scale=False, update_scene_fps=False, update_scene_duration=False, use_cyclic=False, rotate_mode='NATIVE', axis_forward='-Z', axis_up='Y')

    scene = bpy.context.scene

    # part indexes for this sample will start from 0
    partind = 0

    # make subsample bvh for each marked part
    with open(txtfile, "r") as filestream:
        for line in filestream:
            currentline = line.split(',')

            # mark the start and end of the subsample
            start = float(currentline[0])
            end = float(currentline[1])

            # convert to frame number, ZeroEGGS is 60 FPS
            scene.frame_start = int(start * 60)
            scene.frame_end = int(end * 60)

            # export bvh
            bpy.ops.export_anim.bvh(filepath=savepath + 'n_' + str(partcount) + '_o_' + str(mainbvhindex) + '_' + str(partind) + '.bvh',
                check_existing=True,
                filter_glob='*.bvh',
                global_scale=1.0,
                rotate_mode='NATIVE',
                root_transform_only=False)

            partcount += 1
            partind += 1

    mainbvhindex += 1

In [ ]:
# zip and download subsamples for future use
!zip -r zeroeggs_subsamples.zip zeroeggs_subsamples

from google.colab import files
files.download("zeroeggs_subsamples.zip")

In [ ]:
# continue point for unzipping zipped samples, ignore if first time running
!unzip zeroeggs_subsamples.zip

In [ ]:
# clone Bandai repository
!git clone https://github.com/BandaiNamcoResearchInc/Bandai-Namco-Research-Motiondataset.git

In [ ]:
# copy Bandai data to root for ease of access
!mkdir bandai_samples
!cp -r Bandai-Namco-Research-Motiondataset/dataset/Bandai-Namco-Research-Motiondataset-1/data/* bandai_samples

# Eliminating ZeroEGGS Samples

In [ ]:
# install bvhio
!pip install bvhio

In [ ]:
# test bvhio
import bvhio

path = 'zeroeggs_subsamples/n_0_o_1_0.bvh'

root = bvhio.readAsHierarchy(path)
root.printTree()

root.loadRestPose(recursive=True)
print('\nRest pose position and Y-direction of each joint in world space ')
for joint, index, depth in root.layout():
    print(f'{joint.PositionWorld} {joint.UpWorld} {joint.Name}')

In [ ]:
# calculate features from ZeroEGGS subsample BVHs and generate feature vectors as pickles
import os
import time
import pickle
import bvhio
import glm
import numpy as np

feature_size = 21

min_element_size = 1
max_element_size = 16

directory = 'zeroeggs_subsamples'

bodyParts = ['Spine', 'LeftHand', 'RightHand', 'Head', 'RightFoot','LeftFoot', 'RightArm',
             'LeftArm', 'Hips', 'RightForeArm','LeftForeArm', 'Neck', 'LeftLeg','RightLeg',
             'LeftUpLeg', 'RightUpLeg']

def partNameToIndex(partName):
    return bodyParts.index(partName)

def makeVectors(animindex):
    vecs_all = []
    for i in range(min_element_size,max_element_size):
        vecs_all.append(makeVectorOfSize(animindex,i))
    return vecs_all

def makeVectorOfSize(animindex,size):
    arr = np.zeros((feature_size,size))
    joints = jointsall[animindex]
    frame_count = len(joints)

    num_of_frames_per_element = int(frame_count / size)

    for i in range(0,size):
        arr[0,i] = calcAvgDistance(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'LeftHand','RightHand')
        arr[1,i] = calcAvgDistance(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'Head','RightHand')
        arr[2,i] = calcAvgDistance(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'Head','LeftHand')
        arr[3,i] = calcAvgDistance(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'RightFoot','LeftFoot')
        arr[4,i] = calcAvgDistance(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'RightArm','LeftArm')
        arr[5,i] = calcAvgDistance(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'Hips','Head')
        arr[6,i] = calcAvgDistance(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'LeftLeg','RightLeg')
        arr[7,i] = calcAvgDistance(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'RightForeArm','LeftForeArm')
        arr[8,i] = calcAvgDistance(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'Hips','RightHand')
        arr[9,i] = calcAvgDistance(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'Hips','LeftHand')

        arr[10,i] = calcAvgSpeed(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'LeftHand')
        arr[11,i] = calcAvgSpeed(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'RightHand')
        arr[12,i] = calcAvgSpeed(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'LeftForeArm')
        arr[13,i] = calcAvgSpeed(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'RightForeArm')
        arr[14,i] = calcAvgSpeed(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'Head')
        arr[15,i] = calcAvgSpeed(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'Neck')

        arr[16,i] = calcAvgAngle(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'LeftHand','LeftForeArm','LeftArm')
        arr[17,i] = calcAvgAngle(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'RightHand','RightForeArm','RightArm')
        arr[18,i] = calcAvgAngle(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'Spine','Neck','Head')
        arr[19,i] = calcAvgAngle(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'LeftUpLeg','LeftLeg','LeftFoot')
        arr[20,i] = calcAvgAngle(joints,i*num_of_frames_per_element,(i+1)*num_of_frames_per_element,'RightUpLeg','RightLeg','RightFoot')

    return arr

def calcAvgSpeed(joints,pose_no_begin,pose_no_end,joint):
    total = 0
    count = 0
    for i in range(pose_no_begin,pose_no_end-1):
        total += speed(joints,i,i+1,joint)
        count += 1
    if count > 0:
        return total / count
    else:
        return total

def calcAvgDistance(joints,pose_no_begin,pose_no_end,joint_a,joint_b):
    total = 0
    count = 0
    for i in range(pose_no_begin,pose_no_end):
        total += distance(joints,i,joint_a,joint_b)
        count += 1
    if count > 0:
        return total / count
    else:
        return total

def calcAvgAngle(joints,pose_no_begin,pose_no_end,joint_a,joint_cent,joint_b):
    total = 0
    count = 0
    for i in range(pose_no_begin,pose_no_end):
        total += angle(joints,i,joint_a,joint_cent,joint_b)
        count += 1
    if count > 0:
        return total / count
    else:
        return total

def distance(joints,pose_no,joint_a,joint_b):
    return glm.distance(joints[pose_no][partNameToIndex(joint_a)],joints[pose_no][partNameToIndex(joint_b)])

def speed(joints,pose_no_start,pose_no_end,joint_a):
    return glm.distance(joints[pose_no_start][partNameToIndex(joint_a)],joints[pose_no_end][partNameToIndex(joint_a)])

def angle(joints,pose_no,joint_a,joint_cent,joint_b):
    posCent = joints[pose_no][partNameToIndex(joint_cent)]
    v1 = posCent - joints[pose_no][partNameToIndex(joint_a)]
    v2 = posCent - joints[pose_no][partNameToIndex(joint_b)]
    return glm.acos(glm.dot(glm.normalize(v1),glm.normalize(v2)))

# get joint word points to array
joint_per_pose = len(bodyParts)

jointsall = []
counti = 0

!mkdir Pickles

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        print(str(counti) + ' - Processing ' + filename)
        start_time = time.time()

        root = bvhio.readAsHierarchy(f)
        frame_count = len(root.Keyframes)
        joints = np.empty((frame_count,joint_per_pose),dtype=object)
        for frame in range(frame_count):
          loadedPose = root.loadPose(frame)
          for partin,part in enumerate(bodyParts):
            joints[frame][partin] = loadedPose.filter(part)[0].PositionWorld
        jointsall.append(joints)

        vecs = makeVectors(counti)
        file = open('Pickles/' + filename.split('.')[0] + '.pickle', 'wb')
        pickle.dump(vecs, file)
        file.close()
        print("--- %s seconds ---" % (time.time() - start_time))
        counti += 1

In [ ]:
# read from pickles
import pickle

pickleDict = {}

directory = 'Pickles'

for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        file = open(f, 'rb')
        pickleDict[filename.split('.')[0]] = pickle.load(file)
        file.close()

count = 0
for p in pickleDict:
    count += 1
print("Read file count " + str(count))

# normalize per element size and feature
import numpy as np

def getMinVal(fname,element_size,feat):
    vec_of_file = pickleDict[fname]
    vec_of_size = vec_of_file[element_size-1]
    vals = vec_of_size[feat]
    minval = vals[0]
    for i in range(element_size):
        if minval > vals[i]:
            minval = vals[i]
    return minval

def getMaxVal(fname,element_size,feat):
    vec_of_file = pickleDict[fname]
    vec_of_size = vec_of_file[element_size-1]
    vals = vec_of_size[feat]
    maxval = vals[0]
    for i in range(element_size):
        if maxval < vals[i]:
            maxval = vals[i]
    return maxval

def findMaxOf(element_size,feat):
    maxval = getMaxVal(next(iter(pickleDict.keys())),element_size,feat)
    for p in pickleDict:
        maxc = getMaxVal(p,element_size,feat)
        if maxc > maxval:
            maxval = maxc
    return maxval

def findMinOf(element_size,feat):
    minval = getMinVal(next(iter(pickleDict.keys())),element_size,feat)
    for p in pickleDict:
        minc = getMinVal(p,element_size,feat)
        if minc < minval:
            minval = minc
    return minval

def normalizeVec(max_value,min_value,fname,element_size,feat):
    vec_of_file = pickleDict[fname]
    vec_of_size = vec_of_file[element_size-1]
    vals = vec_of_size[feat]
    for i in range(element_size):
        vals[i] = (vals[i] - min_value) / (max_value - min_value)

def normalize(max_value,min_value,element_size,feat):
    for p in pickleDict:
        normalizeVec(max_value,min_value,p,element_size,feat)

max_per_feature_and_size = np.zeros((max_element_size-min_element_size,feature_size))
min_per_feature_and_size = np.zeros((max_element_size-min_element_size,feature_size))

for element_size in range(min_element_size,max_element_size):
    for feat in range(feature_size):
        min_per_feature_and_size[element_size-min_element_size,feat] = findMinOf(element_size,feat)
        max_per_feature_and_size[element_size-min_element_size,feat] = findMaxOf(element_size,feat)
        normalize(max_per_feature_and_size[element_size-min_element_size,feat], min_per_feature_and_size[element_size-min_element_size,feat],element_size,feat)

# validate ranges
mins = 0
maxs = 0
count = 0
for element_size in range(min_element_size,max_element_size):
    for feat in range(feature_size):
        mins += findMinOf(element_size,feat)
        maxs += findMaxOf(element_size,feat)
        count += 1
print(mins/count)
print(maxs/count)

In [ ]:
# make distances matrix for pairwise similarity of subsamples
def distanceBtwFeat(fname1,fname2,element_size,feat):
    vec_of_file1 = pickleDict[fname1]
    vec_of_size1 = vec_of_file1[element_size-1]
    vals1 = vec_of_size1[feat]

    vec_of_file2 = pickleDict[fname2]
    vec_of_size2 = vec_of_file2[element_size-1]
    vals2 = vec_of_size2[feat]

    distance = 0
    count = 0
    for i in range(element_size):
        distance += abs(vals1[i] - vals2[i])
        count += 1
    return (distance / count)

def distanceBtw(fname1,fname2):
    totalDistance = 0
    for element_size in range(min_element_size,max_element_size):
        for feat in range(feature_size):
            totalDistance += distanceBtwFeat(fname1,fname2,element_size,feat)
    return totalDistance

def indexFromFname(fname):
    return int(fname.split('_')[1])

distanceMatrix = np.zeros((len(pickleDict),len(pickleDict)))
fcount = 0

for p1 in pickleDict:
    fcount += 1
    print('calculating for ' + p1 + ' (' + str(fcount) + '/1291)', end='\r')

    for p2 in pickleDict:
        if p1 != p2:
            distanceMatrix[indexFromFname(p1),indexFromFname(p2)] = distanceBtw(p1,p2)
        else:
            distanceMatrix[indexFromFname(p1),indexFromFname(p2)] = float('inf')

# save distance matrix
import pickle
file = open('distanceMatrix.pickle', 'wb')
pickle.dump(distanceMatrix, file)
file.close()

In [ ]:
# load distance matrix for eliminating the similar samples
file = open('distanceMatrix.pickle', 'rb')
distM = pickle.load(file)
file.close()

totalRemaining = len(pickleDict)
targetCount = 100 # we will reduce to 100 samples
eliminateds = []

def find_min_idx(x):
    k = x.argmin()
    ncol = x.shape[1]
    return int(k/ncol), k%ncol

while(totalRemaining > targetCount):
    # find the closest samples
    i,j = find_min_idx(distM)

    # find the one that is closer to others, exluding infinites
    sumi = 0
    sumj = 0
    for n in range(len(pickleDict)):
        if(distM[i,n] != float('inf')):
            sumi += distM[i,n]
        if(distM[j,n] != float('inf')):
            sumj += distM[j,n]

    to_eliminate = i

    if sumi > sumj:
        to_eliminate = j

    # eliminate by setting to infinity
    distM[to_eliminate,:] = float('inf')
    distM[:,to_eliminate] = float('inf')
    eliminateds.append(to_eliminate)
    totalRemaining -=1
    print('Eliminated ' + str(to_eliminate) + ' in comparision btw ' + str(i) + ' - ' + str(j))

# get the remaining ones
remains = []
for i in range(len(pickleDict)):
    if i not in eliminateds:
        remains.append(i)

In [ ]:
# copy remaining ones to different folder
import shutil

!mkdir zeroeggs_remaining_subsamples

def getFnameFromIndex(index):
    search_key = 'n_' + str(index) + '_'
    res = [key for key, val in pickleDict.items() if search_key in key]
    return res[0]

for r in remains:
    shutil.copy2('zeroeggs_subsamples/' + getFnameFromIndex(r) + '.bvh', 'zeroeggs_remaining_subsamples/')

In [ ]:
# zip and download remaining subsamples for future use
!zip -r zeroeggs_remaining_subsamples.zip zeroeggs_remaining_subsamples

from google.colab import files
files.download("zeroeggs_remaining_subsamples.zip")

# Preparing Pose Data For Animation Analysis

In [ ]:
# checkout repository to access ZeroEGGS subsamples
!git clone https://github.com/sinansonlu/animation-personality.git

In [ ]:
# install bvhio
!pip install bvhio

In [ ]:
# prepare numpy arrays using joint world positions of the remaining ZeroEGGS subsamples
import os
import pickle
import bvhio
import numpy as np
from natsort import natsorted

bodyParts_zeroeggs = ['Hips', 'Spine', 'Spine2', 'Neck', 'Head', 'LeftShoulder', 'LeftArm', 'LeftForeArm', 'LeftHand',
            'RightShoulder', 'RightArm', 'RightForeArm', 'RightHand', 'LeftUpLeg', 'LeftLeg', 'LeftFoot',
            'LeftToeBase', 'RightUpLeg', 'RightLeg', 'RightFoot', 'RightToeBase']

joint_per_pose = len(bodyParts_zeroeggs)

directory_zeroeggs = "animation-personality/zeroeggs_remaining_subsamples"
bvhanis_zeroeggs = []

for filename in natsorted(os.listdir(directory_zeroeggs)):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        bvhanis_zeroeggs.append(bvhio.readAsHierarchy(f))

jointsall_zeroeggs = []

for ind, root in enumerate(bvhanis_zeroeggs):
    frame_count = len(root.Keyframes)
    joints = np.empty((frame_count,joint_per_pose),dtype=object)

    for frame in range(frame_count):
        loadedPose = root.loadPose(frame)

        for partin,part in enumerate(bodyParts_zeroeggs):
            joints[frame][partin] = loadedPose.filter(part)[0].PositionWorld

    jointsall_zeroeggs.append(joints)

file = open('zeroeggs_joints.pickle', 'wb')
pickle.dump(jointsall_zeroeggs, file)
file.close()

In [ ]:
# clone Bandai repository
!git clone https://github.com/BandaiNamcoResearchInc/Bandai-Namco-Research-Motiondataset.git

# copy Bandai data to root for ease of access
!mkdir bandai_samples
!cp -r Bandai-Namco-Research-Motiondataset/dataset/Bandai-Namco-Research-Motiondataset-1/data/* bandai_samples

In [ ]:
# prepare numpy arrays using joint world positions of the Bandai samples
import os
import pickle
import bvhio
import numpy as np
from natsort import natsorted

animation_names =[ # bow
    "bow_active", # 0
    "bow_angry", # 1
    "bow_childish", # 2
    "bow_chimpira", # 3
    "bow_feminine", # 4
    "bow_giant", # 5
    "bow_happy", # 6
    "bow_masculinity", # 7
    "bow_musical", # 8
    "bow_normal", # 9
    "bow_not-confident", # 10
    "bow_old", # 11
    "bow_proud", # 12
    "bow_sad", # 13
    "bow_tired", # 14
    # bye
    "bye_active", # 0
    "bye_angry", # 1
    "bye_childish", # 2
    "bye_chimpira", # 3
    "bye_feminine", # 4
    "bye_giant", # 5
    "bye_happy", # 6
    "bye_masculinity", # 7
    "bye_musical", # 8
    "bye_normal", # 9
    "bye_not-confident", # 10
    "bye_old", # 11
    "bye_proud", # 12
    "bye_sad", # 13
    "bye_tired", # 14
    # byebye
    "byebye_active", # 0
    "byebye_angry", # 1
    "byebye_childish", # 2
    "byebye_chimpira", # 3
    "byebye_feminine", # 4
    "byebye_giant", # 5
    "byebye_happy", # 6
    "byebye_masculinity", # 7
    "byebye_musical", # 8
    "byebye_normal", # 9
    "byebye_not-confident", # 10
    "byebye_old", # 11
    "byebye_proud", # 12
    "byebye_sad", # 13
    "byebye_tired", # 14
    # dash
    "dash_active", # 0
    "dash_angry", # 1
    "dash_childish", # 2
    "dash_chimpira", # 3
    "dash_feminine", # 4
    "dash_giant", # 5
    "dash_happy", # 6
    "dash_masculinity", # 7
    "dash_musical", # 8
    "dash_normal", # 9
    "dash_not-confident", # 10
    "dash_old", # 11
    "dash_proud", # 12
    "dash_sad", # 13
    "dash_tired", # 14
    # guide
    "guide_active", # 0
    "guide_angry", # 1
    "guide_childish", # 2
    "guide_chimpira", # 3
    "guide_feminine", # 4
    "guide_giant", # 5
    "guide_happy", # 6
    "guide_masculinity", # 7
    "guide_musical", # 8
    "guide_normal", # 9
    "guide_not-confident", # 10
    "guide_old", # 11
    "guide_proud", # 12
    "guide_sad", # 13
    "guide_tired", # 14
    # run
    "run_active", # 0
    "run_angry", # 1
    "run_childish", # 2
    "run_chimpira", # 3
    "run_feminine", # 4
    "run_giant", # 5
    "run_happy", # 6
    "run_masculinity", # 7
    "run_musical", # 8
    "run_normal", # 9
    "run_not-confident", # 10
    "run_old", # 11
    "run_proud", # 12
    "run_sad", # 13
    "run_tired", # 14
     # walk
    "walk_active_1", # 0
    "walk_angry_1", # 1
    "walk_childish_1", # 2
    "walk_chimpira_1", # 3
    "walk_feminine_1", # 4
    "walk_giant_1", # 5
    "walk_happy_1", # 6
    "walk_masculinity_1", # 7
    "walk_musical_1", # 8
    "walk_normal_1", # 9
    "walk_not-confident_1", # 10
    "walk_old_1", # 11
    "walk_proud_1", # 12
    "walk_sad_1", # 13
    "walk_tired_1", # 14
    "walk_active_2", # 0
    "walk_angry_2", # 1
    "walk_childish_2", # 2
    "walk_chimpira_2", # 3
    "walk_feminine_2", # 4
    "walk_giant_2", # 5
    "walk_happy_2", # 6
    "walk_masculinity_2", # 7
    "walk_musical_2", # 8
    "walk_normal_2", # 9
    "walk_not-confident_2", # 10
    "walk_old_2", # 11
    "walk_proud_2", # 12
    "walk_sad_2", # 13
    "walk_tired_2", # 14
    # other
    "call_normal_1",
    "call_normal_2",
    "kick_normal",
    "punch_normal_1",
    "punch_normal_2",
    "respond_normal",
    "slash_normal_1",
    "slash_normal_2"
]

directory_bandai = "bandai_samples/"

def toFileName(aniName):
    if(aniName[-2:] == '_1'):
        return directory_bandai + 'dataset-1_' + aniName[:-2] + '_001.bvh'
    elif(aniName[-2:] == '_2'):
        return directory_bandai + 'dataset-1_' + aniName[:-2] + '_002.bvh'
    else:
        return directory_bandai + 'dataset-1_' + aniName + '_001.bvh'

bodyParts_bandai = ['Hips', 'Spine', 'Chest', 'Neck', 'Head', 'Shoulder_L', 'UpperArm_L', 'LowerArm_L', 'Hand_L',
            'Shoulder_R', 'UpperArm_R', 'LowerArm_R', 'Hand_R', 'UpperLeg_L', 'LowerLeg_L', 'Foot_L',
            'Toes_L', 'UpperLeg_R', 'LowerLeg_R', 'Foot_R', 'Toes_R']

joint_per_pose = len(bodyParts_bandai)

bvhanis_bandai = []

for aniName in animation_names:
    bvhanis_bandai.append(bvhio.readAsHierarchy(toFileName(aniName)))

jointsall_bandai = []

for ind, root in enumerate(bvhanis_bandai):
    frame_count = len(root.Keyframes)
    joints = np.empty((frame_count,joint_per_pose),dtype=object)

    for frame in range(frame_count):
        loadedPose = root.loadPose(frame)

        for partin,part in enumerate(bodyParts_bandai):
            joints[frame][partin] = loadedPose.filter(part)[0].PositionWorld

    jointsall_bandai.append(joints)

file = open('bandai_joints.pickle', 'wb')
pickle.dump(jointsall_bandai, file)
file.close()